Proyecto Final
==============

### Tratamiento de Datos
### Máster de Ing. de Telecomunicación

## Autores

Juan Manuel Espinosa Moral ([100406523@alumnos.uc3m.es](mailto:100406523@alumnos.uc3m.es))

José Manuel García Núñez ([100544621@alumnos.uc3m.es](mailto:100544621@alumnos.uc3m.es))

In [ ]:
# Integración en Collab

# Librerías de drive
from google.colab import drive
import os, sys

# Montaje
drive.mount("/content/drive")

# Directorio actual
print(os.getcwd())

# Cambio de directorio al compartido
directory_path = "/content/drive/MyDrive/Colab Notebooks/proyecto_td/"  # path
# If para crear el directorio en su path en caso de no existir
if not os.path.exists(directory_path):
  os.makedirs(directory_path)
  print(f"Directory created: {directory_path}")

os.chdir(directory_path) # switch de directorio

## 1. Análisis de Variables de Entrada

- Carga del dataset: datos del archivo JSON.
- Categorías: las más frecuentes.
- Rating y visualizaciones.
- Análisis de correlación: categorias y variables de salida.

In [39]:
# Create directory to store results
if not os.path.exists("data/section_1/"):
  os.makedirs("data/section_1/")
  print(f"Directory created: {'data/section_1/'}")

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Cargar el JSON
if not os.path.exists("full_format_recipes.json"):
  df = pd.read_json("data/full_format_recipes.json")
else:
  df = pd.read_json("full_format_recipes.json")

# Explorar categories en cuanto a aparición
category_counts = df['categories'].explode().value_counts()
print("Categorías más frecuentes:\n", category_counts.head())

# Top 10 categories
category_counts.head(10).plot(kind='bar', title="Frecuencia de las categorías")
plt.ylabel("Número de recetas")
plt.savefig("data/section_1/no_recipes.png")
plt.show()

# Analizar la relación entre categorías y ratings, filtrando por las más frecuentes
df_exploded = df.explode('categories')  # Expandir listas de categorías
df_exploded = df_exploded.reset_index(drop=True)  # Resetear el índice

# Filtrar por las 10 categorías con mejor rating
top_categories = category_counts.head(10).index.tolist()
df_filtered = df_exploded[df_exploded['categories'].isin(top_categories)]

plt.figure(figsize=(12, 6))
sns.boxplot(x='categories', y='rating', data=df_filtered)
plt.xticks(rotation=90)
plt.title("Distribución de ratings por categoría (Top 10)")
plt.savefig("data/section_1/ratings_distribution_top10.png")
plt.show()

In [ ]:
# correlacion y heatmap de variables
correlation = df[['fat', 'protein', 'calories', 'sodium', 'rating']].corr()

print(correlation)

sns.heatmap(correlation, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.savefig("data/section_1/corr.png")
plt.show()

In [ ]:
# Relación de ratings por categories
df_exploded = df.explode('categories')  # Expandir listas de categorías
df_exploded = df_exploded.reset_index(drop=True)
plt.figure(figsize=(120, 6))
sns.boxplot(x='categories', y='rating', data=df_exploded)
plt.xticks(rotation=90)
plt.title("Distribución de ratings por categoría")
plt.savefig("data/section_1/ratings_distribution.png")
plt.show()

In [ ]:
# import matplotlib.pyplot as plt

# Analisis logarítmico relacionando fat y rating
plt.scatter(df['fat'], df['rating'])
plt.xscale('log')  # Apply logarithmic scale to x-axis
plt.xlabel('Fat (grams) - Log Scale')
plt.ylabel('Rating')
plt.title('Fat vs. Rating (Log Scale)')
plt.savefig("data/section_1/ratings_vs_calories_log.png")
plt.show()

In [ ]:
# Relación entre rating y calories

# se definen los tramos de calorias
bins = [0, 200, 400, 600, 800, 1000, float('inf')]
labels = ['0-200', '201-400', '401-600', '601-800', '801-1000', '1001+']

# creación de una columna independiente con calorías
df['calorie_bins'] = pd.cut(df['calories'], bins=bins, labels=labels)

# creación de una medía de calorias en baase al rating
average_ratings = df.groupby('calorie_bins')['rating'].mean()

# graficación de resultados
plt.bar(average_ratings.index, average_ratings.values)
plt.xlabel('Calorie Range')
plt.ylabel('Average Rating')
plt.title('Average Rating vs. Calories')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig("data/section_1/ratings_vs_calories.png")
plt.show()

In [ ]:
# se definen los tramos de grasa (gramos)
bins_fat = [0, 10, 20, 30, 40, 50, float('inf')]
labels_fat = ['0-10', '11-20', '21-30', '31-40', '41-50', '51+']

# creación de una columna independiente con grasas
df['fat_bins'] = pd.cut(df['fat'], bins=bins_fat, labels=labels_fat)

# creación de una media de calorias en base a la cantidad de grasas
average_ratings = df.groupby('fat_bins')['rating'].mean()

# graficación de resultados
plt.bar(average_ratings.index, average_ratings.values)
plt.xlabel('Fat Range')
plt.ylabel('Average Rating')
plt.title('Average Rating vs. Fat')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig("data/section_1/ratings_vs_fat.png")
plt.show()

In [ ]:
# se definen los tramos de proteina (gramos)
bins_protein = [0, 10, 20, 30, 40, 50, float('inf')]
labels_protein = ['0-10', '11-20', '21-30', '31-40', '41-50', '51+']

# creación de una columna independiente con proteina
df['protein_bins'] = pd.cut(df['protein'], bins=bins_protein, labels=labels_protein)

# creación de una media de calorias en base a la cantidad de proteina
average_ratings = df.groupby('protein_bins')['rating'].mean()

# graficación de resultados
plt.bar(average_ratings.index, average_ratings.values)
plt.xlabel('Protein Range')
plt.ylabel('Average Rating')
plt.title('Average Rating vs. Protein')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig("data/section_1/ratings_vs_protein.png")
plt.show()

In [ ]:
# Esta parte no se si tiene mucho sentido mantener

top_20_categories = df_exploded['categories'].value_counts().head(20).index.tolist()
df_filtered = df_exploded[df_exploded['categories'].isin(top_20_categories)]

variables = ['fat', 'protein', 'calories', 'sodium', 'rating']
for variable in variables:
    plt.figure(figsize=(12, 6))
    sns.boxplot(x='categories', y=variable, data=df_filtered)
    plt.xticks(rotation=90)
    plt.title(f'{variable.capitalize()} Distribution by Top 20 Categories')
    plt.tight_layout()
    plt.show()

## 2. Implementación de un *pipeline* para el preprocesado de los textos

In [ ]:
# Create directory to store results
if not os.path.exists("data/section_2/"):
  os.makedirs("data/section_2/")
  print(f"Directory created: {'data/section_2/'}")

In [ ]:
import re
import spacy

# cargado de spaCy
nlp = spacy.load("en_core_web_sm")

# definición del método de procesado de texto del pipeline
def preprocess_text(text, use_spacy=False):

    # Si el texto es NaN o no es string/list, convertirlo a cadena vacía
    if not isinstance(text, (str, list)):
        text = ''

    # si el texto es una lista, unir sus elementos en una sola cadena
    if isinstance(text, list):
        text = ' '.join(text)

    # pasar a minúsculas
    text = text.lower()

    # Eliminar caracteres especiales y números
    text = re.sub(r'[^a-z\s]', '', text)

    if use_spacy:
        # Usar SpaCy para tokenización y lematización
        doc = nlp(text)
        tokens = [token.lemma_ for token in doc if not token.is_stop]
    else:
        # Tokenizar el texto
        tokens = nltk.word_tokenize(text)

        # Eliminar stopwords
        stop_words = set(stopwords.words('english'))
        tokens = [word for word in tokens if word not in stop_words]

        # Lematizar
        lemmatizer = nltk.WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Unir tokens de nuevo en un solo string
    processed_text = ' '.join(tokens)
    return processed_text

# Para "categories"

# Manejar valores NaN en la columna 'categories'
df['categories'] = df['categories'].fillna('')

# Aplicar el pipeline al conjunto de datos
df['processed_categories'] = df['categories'].apply(preprocess_text, use_spacy=True)

# print de resultados
print(df[['categories', 'processed_categories']].head())

# Para "desc"

# Manejar valores NaN en la columna 'categories'
df['desc'] = df['desc'].fillna('')

# Aplicar el pipeline al conjunto de datos
df['processed_desc'] = df['desc'].apply(preprocess_text, use_spacy=True)

# print de resultados
print(df[['desc', 'processed_desc']].head())

# Guardamos el resultado
df.to_csv("data/section_2/df_proccessed.csv", index=False)

## 3. Representación vectorial de los documentos mediante tres procedimientos diferentes

In [ ]:
# Create directory to store results
if not os.path.exists("data/section_3/"):
  os.makedirs("data/section_3/")
  print(f"Directory created: {'data/section_3/'}")

### Procedimiento 1: TF-IDF

In [ ]:
# Bibliotecas
from sklearn.feature_extraction.text import TfidfVectorizer

# Para "categories"

# Creación del objeto a vectorizar
vectorizer = TfidfVectorizer()

# conversión de datos para vectorizar
tfidf_matrix1 = vectorizer.fit_transform(df['processed_categories'])

# extracción de los nombres
feature_names1 = vectorizer.get_feature_names_out()

# creación del dataframe con los datos convertidos a arrays pasados por el método y los nombres extraidos
tfidf_df1 = pd.DataFrame(tfidf_matrix1.toarray(), columns=feature_names1, index=df.index)

# Guardamos el resultado
tfidf_df1.to_csv("data/section_3/df_categories_with_tfidf.csv", index=False)

# print de los resultados
print("TF-IDF DataFrame for 'processed_categories':")
print(tfidf_df1)

# Para "desc"

# Creación del objeto a vectorizar
vectorizer2 = TfidfVectorizer()  # Or use: vectorizer = TfidfVectorizer() if reusing

# conversión de datos para vectorizar
tfidf_matrix2 = vectorizer2.fit_transform(df['processed_desc'])  # Or use: vectorizer.fit_transform if reusing

# extracción de los nombres
feature_names2 = vectorizer2.get_feature_names_out()  # Or use: vectorizer.get_feature_names_out() if reusing

# creación del dataframe con los datos convertidos a arrays pasados por el método y los nombres extraidos
tfidf_df2 = pd.DataFrame(tfidf_matrix2.toarray(), columns=feature_names2, index=df.index)

# print de los resultados
print("\nTF-IDF DataFrame for 'processed_desc':")
print(tfidf_df2)

# Guardamos el resultado
tfidf_df2.to_csv("data/section_3/df_desc_with_tfidf.csv", index=False)

### Procedimiento 2: *Word2Vec*

In [ ]:
# Bibliotecas
!pip install nltk==3.8.1
!pip install gensim

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import numpy as np
from gensim.models import Word2Vec

# Método para obtener vectores promediados
def get_vector(text_words, model):
  """Obtiene el vector promedio para un texto."""
  vectors = [model.wv[word] for word in text_words if word in model.wv]
  if vectors:
    return np.mean(vectors, axis=0)
  else:
    return np.zeros(model.vector_size)  # Vector de ceros si no hay palabras en el vocabulario

# Word2Vec para "categories"

# 1. Preparar datos de texto para 'categories'
sentences1 = [row.split() for row in df['processed_categories']]

# 2. Entrenar el modelo para 'categories'
model1 = Word2Vec(sentences1, vector_size=100, window=5, min_count=5, workers=4)

# 3. Obtener vectores promediados para 'categories'
df['category_vector'] = df['processed_categories'].apply(lambda x: get_vector(x.split(), model1))

# Muestra 5 resultados
print("Resultados para 'categories':")
for index in range(5):
  print(f"Fila {index}: {df['category_vector'][index][:5]}")

# Word2Vec para "desc"

# 1. Preparar datos de texto para 'categories'
sentences2 = [row.split() for row in df['processed_desc']]

# 2. Entrenar el modelo para 'categories'
model2 = Word2Vec(sentences2, vector_size=100, window=5, min_count=5, workers=4)

# 3. Obtener vectores promediados para 'categories'
df['desc_vector'] = df['processed_desc'].apply(lambda x: get_vector(x.split(), model2))

# Muestra 5 resultados
print("\nResultados para 'desc':")
for index in range(5):
  print(f"Fila {index}: {df['desc_vector'][index][:5]}")

# Guardamos el resultado
df.to_csv("data/section_3/df_with_word2vec.csv", index=False)

### **DEBUG**: *Testing* de los metodos de vectorización TF-IDF y vec2sec

In [ ]:
from wordcloud import WordCloud
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA # Import PCA
import numpy as np # Import numpy

# Visualización de TF-IDF con Nube de Palabras
word_weights = dict(zip(feature_names1, tfidf_matrix1.sum(axis=0).tolist()[0]))
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_weights)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Nube de Palabras TF-IDF")
plt.savefig("data/section_3/tf_idf_wordcloud.png")
plt.show()

# Visualización de Word2Vec con PCA
pca = PCA(n_components=2)
vectors_2d = pca.fit_transform(df['category_vector'].to_list())
plt.figure(figsize=(10, 10))
plt.scatter(vectors_2d[:, 0], vectors_2d[:, 1])
plt.title("Visualización de vectores Word2Vec con PCA")
plt.xlabel("Componente Principal 1")
plt.ylabel("Componente Principal 2")
plt.savefig("data/section_3/word2vec_pca.png")
plt.show()

# Visualización de Word2Vec con t-SNE

tsne = TSNE(n_components=2, random_state=42)
# Convert the list of vectors to a 2D NumPy array
category_vectors = df['category_vector'].to_list()
vectors_2d = tsne.fit_transform(np.array(category_vectors).reshape(len(category_vectors), -1)) # Reshape if necessary
plt.figure(figsize=(10, 10))
plt.scatter(vectors_2d[:, 0], vectors_2d[:, 1])
plt.title("Visualización de vectores Word2Vec con t-SNE")
plt.xlabel("Dimensión 1")
plt.ylabel("Dimensión 2")
plt.savefig("data/section_3/word2vec_tsne.png")
plt.show()

### Procedimiento 3: *Embeddings* contextuales calculados a partir de modelos basados en *transformers*

#### Sin gradientes

In [ ]:
# Import de Pytorch y BERT
from transformers import BertTokenizer, BertModel
import torch

# Usar GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Reducir el tamaño del dataset a un 10%
df_sampled = df.sample(frac=0.1, random_state=42)

# Cargar el modelo y el tokenizer de BERT
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

# Función para generar embeddings contextuales
def generate_bert_embeddings(text):
    # Tokenizar el texto
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Desactivar gradientes
    with torch.no_grad():
        outputs = model(**inputs)

    # extraer embeddings
    embeddings = outputs.last_hidden_state[:, 0, :].squeeze(0).cpu().numpy()
    return embeddings

# Aplicar BERT a las columnas y conversión de los embeddings en listas
df_sampled['bert_embeddings_categories'] = df_sampled['processed_categories'].apply(lambda x: generate_bert_embeddings(x).tolist())
df_sampled['bert_embeddings_desc'] = df_sampled['processed_desc'].apply(lambda x: generate_bert_embeddings(x).tolist())

# Guardar el resultado en un archivo CSV (opcional)
df_sampled.to_csv("data/section_3/df_with_bert_embeddings.csv", index=False)

# visualizar algunos registros
print(df_sampled[['processed_categories', 'bert_embeddings_categories']].head())
print(df_sampled[['processed_desc', 'bert_embeddings_desc']].head())


#### Con gradientes

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# Usar GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Reducir el tamaño del dataset para pruebas (opcional)
df_sampled = df.sample(frac=0.1, random_state=42)

# Cargar el modelo y el tokenizer de BERT
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

# Configurar el modelo en modo entrenamiento
model.train()

# Configurar un optimizador
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

def generate_bert_embeddings_with_gradients(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    cls_embeddings = outputs.last_hidden_state[:, 0, :]
    cls_embeddings.retain_grad()
    loss = cls_embeddings.norm()
    loss.backward()
    gradients = cls_embeddings.grad.detach().cpu().numpy()
    return cls_embeddings.detach().cpu().numpy(), gradients

# Aplicar BERT con gradientes a 'processed_categories'
df_sampled['bert_embeddings_categories'] = df_sampled['processed_categories'].apply(
    lambda x: generate_bert_embeddings_with_gradients(x)
)

# Aplicar BERT con gradientes a 'processed_desc'
df_sampled['bert_embeddings_desc'] = df_sampled['processed_desc'].apply(
    lambda x: generate_bert_embeddings_with_gradients(x)
)

# Guardar el resultado en un archivo CSV (opcional)
df_sampled.to_csv("data/section_3/df_with_bert_embeddings.csv", index=False)

# Visualizar los resultados
print(df_sampled[['processed_categories', 'bert_embeddings_categories', 'processed_desc', 'bert_embeddings_desc']].head())

## 4. Entrenamiento y evaluación de modelos de regresión

### Estrategia 1: Redes neuronales utilizando PyTorch

#### Paso 1: Bibliotecas y configuraciones

In [ ]:
# Bibliotecas de redes neuronales
import torch.nn as nn
import torch.nn.functional as F

# Biblioteca de optimización y función de pérdida
import torch.optim as optim
from torch.optim import lr_scheduler

# Usar GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

#### Paso 2: Definir la red

In [ ]:
# TODO: La siguiente clase se basa en LeNet -> Reformular para texto
class Net(nn.Module):

    # En el inicializador vamos a especificar los bloques de cómputo que tienen parámetros a definir, es decir, las capas que serán necesarias. Los definimos de forma independiente (sin unir), pues de momento son bloques aislados y no forman una red.
    def __init__(self):

        super(Net, self).__init__()

        # Capa convolucional:
        #	- Canales in = 3 (la imagen de entrada tiene 3 canales RGB)
        #	- 6 canales out
        #	- Filtro de tamaño 5x5
        self.conv1 = nn.Conv2d(3, 6, 5)

        # Capa de Maxpooling con tamaño 2x2
        self.pool = nn.MaxPool2d(2, 2)

        # Capa convolucional:
        #	- Canales in = 6 (de la capa anterior)
        #	- 16 canales out
        #	- Filtro de tamaño 5x5
        self.conv2 = nn.Conv2d(6, 16, 5)

        # Capa completamente conectada (y = Wx + b):
        #	- Canales in = 16 * 5 * 5 (16 capa anterior, 5x5 es la dimensión de la imagen que llega a esta capa)
        #	- Canales out = 120
        self.fc1 = nn.Linear(16 * 5 * 5, 120)

        # Capa completamente conectada (y = Wx + b)
        #	- Canales in = 120 (capa anterior)
        #	- Canales out = 84
        self.fc2 = nn.Linear(120, 84)

        # Capa completamente conectada (y = Wx + b)
        #	- Canales in = 84 (capa anterior)
        #	- Canales out = 10 (tenemos 10 dígitos a clasificar)
        self.fc3 = nn.Linear(84, 10)

    # En "forward" definimos la estructura de la red a través de su grafo computacional. Es donde conectamos los bloques antes definidos y metemos otros más simples.
    def forward(self, x):

        # Entrada -> conv1 -> activación relu -> Max pooling sobre una ventana (2, 2) -> x
        x = self.pool(F.relu(self.conv1(x)))

        # x -> conv2 -> activación relu -> Max pooling sobre una ventana (2, 2) -> x
        x = self.pool(F.relu(self.conv2(x)))

        # Cambiamos la forma del tensor para vectorizarlo (16x6x6 -> 120) -> x
        x = x.view(-1, 16 * 5 * 5)

        # x -> fc1 -> relu -> x
        x = F.relu(self.fc1(x))

        # x -> fc2 -> relu -> x
        x = F.relu(self.fc2(x))

        # fc3
        x = self.fc3(x)

        return x

# Invocamos el constructor de la red (método init())
net = Net()

# Pasamos la red al dispositivo que estemos usando (GPU)
net.to(device)

# Información de la red

## Obtenemos la lista
params = list(net.parameters())

## Número de parámetros
print("Número de parámetros de la red {:d}".format(len(params)))

## Tamaño de los parámetros:
for param in params:
    print(param.size())

#### Entrenamiento con los valores de TF-IDF

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd

# Preparación de los datos

# Cargar los datos
tfidf_df1 = pd.read_csv("data/section_3/df_categories_with_tfidf.csv")
tfidf_df2 = pd.read_csv("data/section_3/df_desc_with_tfidf.csv")

# Combinar las representaciones TF-IDF generadas
data_combined = pd.concat([tfidf_df1, tfidf_df2], axis=1)

# Verificar y manejar valores NaN en los datos
data_combined = data_combined.fillna(0)  # Rellenar NaN con 0
y = df['rating'].fillna(0).values  # Rellenar NaN en el target con 0

# Definir las etiquetas (target) y características (features)
X = data_combined.values

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalización de los datos
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)
std[std == 0] = 1  # Evitar división por cero
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

# Eliminar posibles valores NaN o infinitos después de la normalización
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

# Convertir los datos a tensores de PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Definición del modelo
class RegressionModel(nn.Module):
    def __init__(self, input_size):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

# Instanciar el modelo
input_size = X_train.shape[1]
model = RegressionModel(input_size)

# Definir la función de pérdida y el optimizador
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento del modelo
epochs = 50
batch_size = 32
num_batches = len(X_train_tensor) // batch_size

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for i in range(num_batches):
        start = i * batch_size
        end = start + batch_size
        inputs = X_train_tensor[start:end]
        targets = y_train_tensor[start:end]

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass y optimización
        optimizer.zero_grad()
        loss.backward()

        # Clip de gradientes para evitar explosión
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/num_batches:.4f}")

# Evaluación del modelo
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor).numpy()
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)

print(f"MSE: {mse:.4f}")
print(f"R^2: {r2:.4f}")



Epoch [1/5], Loss: 5.1733
Epoch [2/5], Loss: 1.6628
Epoch [3/5], Loss: 1.2356
Epoch [4/5], Loss: 1.1136
Epoch [5/5], Loss: 0.9447
MSE: 2.5845
R^2: -0.4950


#### Entrenamiento con los valores de *Word2Vec*

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Cargar los datos
df = pd.read_csv("data/section_3/df_with_word2vec.csv")

# Convertir los vectores de categorías y ratings a tensores de PyTorch
X = np.stack(df['category_vector'].values)
y = df['rating'].values

# Manejar valores NaN en los datos
X = np.nan_to_num(X)  # Reemplazar NaN por 0
y = np.nan_to_num(y)

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertir los datos a tensores de PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Definir la red neuronal
class RegressionModel(nn.Module):
    def __init__(self, input_size):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

# Inicializar el modelo, la función de pérdida y el optimizador
input_size = X_train.shape[1]
model = RegressionModel(input_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Reducir la tasa de aprendizaje

# Entrenamiento del modelo
epochs = 100
for epoch in range(epochs):
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Verificar si la pérdida es NaN o infinita
    if torch.isnan(loss) or torch.isinf(loss):
        print(f"Stopping training at epoch {epoch + 1} due to NaN/infinite loss.")
        break

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")

# Evaluación del modelo
model.eval()
with torch.no_grad():
    y_pred_train = model(X_train_tensor).numpy()
    y_pred_test = model(X_test_tensor).numpy()

    train_mse = mean_squared_error(y_train, y_pred_train)
    test_mse = mean_squared_error(y_test, y_pred_test)

print(f"Train MSE: {train_mse:.4f}")
print(f"Test MSE: {test_mse:.4f}")


Epoch [10/100], Loss: 15.3623
Epoch [20/100], Loss: 15.1992
Epoch [30/100], Loss: 15.0344
Epoch [40/100], Loss: 14.8671
Epoch [50/100], Loss: 14.6948
Epoch [60/100], Loss: 14.5119
Epoch [70/100], Loss: 14.3104
Epoch [80/100], Loss: 14.0764
Epoch [90/100], Loss: 13.8234
Epoch [100/100], Loss: 13.5528
Train MSE: 13.5245
Test MSE: 13.5902


#### Entrenamiento con los valores de *embeddings* contextuales

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Cargar los datos
df = pd.read_csv("data/section_3/df_with_bert_embeddings.csv")

# Convertir los vectores de categorías y ratings a tensores de PyTorch
X = np.stack(df['category_vector'].values)
y = df['rating'].values

# Manejar valores NaN en los datos
X = np.nan_to_num(X)  # Reemplazar NaN por 0
y = np.nan_to_num(y)

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertir los datos a tensores de PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Definir la red neuronal
class RegressionModel(nn.Module):
    def __init__(self, input_size):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

# Inicializar el modelo, la función de pérdida y el optimizador
input_size = X_train.shape[1]
model = RegressionModel(input_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Reducir la tasa de aprendizaje

# Entrenamiento del modelo
epochs = 100
for epoch in range(epochs):
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Verificar si la pérdida es NaN o infinita
    if torch.isnan(loss) or torch.isinf(loss):
        print(f"Stopping training at epoch {epoch + 1} due to NaN/infinite loss.")
        break

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")

# Evaluación del modelo
model.eval()
with torch.no_grad():
    y_pred_train = model(X_train_tensor).numpy()
    y_pred_test = model(X_test_tensor).numpy()

    train_mse = mean_squared_error(y_train, y_pred_train)
    test_mse = mean_squared_error(y_test, y_pred_test)

print(f"Train MSE: {train_mse:.4f}")
print(f"Test MSE: {test_mse:.4f}")


Epoch [10/100], Loss: 15.3623
Epoch [20/100], Loss: 15.1992
Epoch [30/100], Loss: 15.0344
Epoch [40/100], Loss: 14.8671
Epoch [50/100], Loss: 14.6948
Epoch [60/100], Loss: 14.5119
Epoch [70/100], Loss: 14.3104
Epoch [80/100], Loss: 14.0764
Epoch [90/100], Loss: 13.8234
Epoch [100/100], Loss: 13.5528
Train MSE: 13.5245
Test MSE: 13.5902
